In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
bitstamp_file = "resources/bitstampUSD_1-min_data_2012-01-01_to_2019-08-12.csv"
bitstamp_df = pd.read_csv(bitstamp_file)
bitstamp_df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
coinbase_file = "resources/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv"
coinbase_df = pd.read_csv(coinbase_file)
coinbase_df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1417411980,300.0,300.0,300.0,300.0,0.01,3.0,300.0
1,1417412040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1417412100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1417412160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1417412220,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Transform bitstamp DataFrame

In [21]:
# Create a copy of the dataframe
bitstamp_transform = bitstamp_df.copy()

# Drop NaN rows
bitstamp_transform = bitstamp_transform.dropna(thresh=2)

# Rename columns to acceptable values within postgres
bitstamp_transform.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume_btc', 'volume_currency', 'weighted_price']

# Create a new index to as the id
bitstamp_transform = bitstamp_transform.reset_index()
bitstamp_transform.columns['index'] = 'id'
bitstamp_transform['id'] = bitstamp_transform.index

# Display database
bitstamp_transform

TypeError: Index does not support mutable operations

### Transform coinbase DataFrame

In [12]:
# Create a copy of the dataframe
coinbase_transform = coinbase_df.copy()

# Drop NaN rows
coinbase_transform = coinbase_transform.dropna(thresh=2)

# Rename columns to acceptable values within postgres
coinbase_transform.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume_btc', 'volume_currency', 'weighted_price']
coinbase_transform

,timestamp,open,high,low,close,volume_btc,volume_currency,weighted_price
0,1417411980,300.00,300.00,300.00,300.00,0.010000,3.000000,300.000000
7,1417412400,300.00,300.00,300.00,300.00,0.010000,3.000000,300.000000
51,1417415040,370.00,370.00,370.00,370.00,0.010000,3.700000,370.000000
77,1417416600,370.00,370.00,370.00,370.00,0.026556,9.825550,370.000000
1436,1417498140,377.00,377.00,377.00,377.00,0.010000,3.770000,377.000000
...,...,...,...,...,...,...,...,...
2099755,1546898520,4006.01,4006.57,4006.00,4006.01,3.382954,13553.433078,4006.390309
2099756,1546898580,4006.01,4006.57,4006.00,4006.01,0.902164,3614.083169,4006.017233
2099757,1546898640,4006.01,4006.01,4006.00,4006.01,1.192123,4775.647308,4006.003635
2099758,1546898700,4006.01,4006.01,4005.50,4005.50,2.699700,10814.241898,4005.719991


### Create database connection

In [13]:
connection_string = "postgres:password@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [14]:
engine.table_names()

['bitstamp', 'coinbase']

### Load Dataframes into database

In [16]:
bitstamp_transform.to_sql(name='bitstamp', con=engine, if_exists='append')

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "index" of relation "bitstamp" does not exist
LINE 1: INSERT INTO bitstamp (index, timestamp, open, high, low, clo...
                              ^

[SQL: INSERT INTO bitstamp (index, timestamp, open, high, low, close, volume_btc, volume_currency, weighted_price) VALUES (%(index)s, %(timestamp)s, %(open)s, %(high)s, %(low)s, %(close)s, %(volume_btc)s, %(volume_currency)s, %(weighted_price)s)]
[parameters: ({'index': 0, 'timestamp': 1325317920, 'open': 4.39, 'high': 4.39, 'low': 4.39, 'close': 4.39, 'volume_btc': 0.45558087, 'volume_currency': 2.0000000193000003, 'weighted_price': 4.39}, {'index': 478, 'timestamp': 1325346600, 'open': 4.39, 'high': 4.39, 'low': 4.39, 'close': 4.39, 'volume_btc': 48.0, 'volume_currency': 210.72, 'weighted_price': 4.39}, {'index': 547, 'timestamp': 1325350740, 'open': 4.5, 'high': 4.57, 'low': 4.5, 'close': 4.57, 'volume_btc': 37.86229723, 'volume_currency': 171.38033753, 'weighted_price': 4.526

In [ ]:
coinbase_transform.to_sql(name='coinbase', con=engine, if_exists='append', index=True)